## NN for text summarization
seq2seq model that can create relevant summaries

https://cs224d.stanford.edu/reports/lucilley.pdf for abstractive summary

http://kavita-ganesan.com/opinosis-opinion-dataset/#.Xt_Mq2pKjUJ but it's java.

or try: https://arxiv.org/pdf/1911.02247.pdf
trained on yelp and amazon reviews, but there's no pre-trained..

In [1]:
import matplotlib
# Specify renderer
# matplotlib.use('Agg')

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Boiler-plate settings for producing pub-quality figures
# 1 point = 1/72 inch
from cycler import cycler
matplotlib.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
matplotlib.rcParams.update({
    'figure.figsize': (8, 5)  # inches
    ,
    'font.size':
    22  # points
    ,
    'legend.fontsize':
    16  # points
    ,
    'lines.linewidth':
    1.5  # points
    ,
    'axes.linewidth':
    1.5  # points
    ,
    'text.usetex':
    True  # Use LaTeX to layout text
    ,
    'font.family':
    "serif"  # Use serifed fonts
    ,
    'xtick.major.size':
    10  # length, points
    ,
    'xtick.major.width':
    1.5  # points
    ,
    'xtick.minor.size':
    6  # length, points
    ,
    'xtick.minor.width':
    1  # points
    ,
    'ytick.major.size':
    10  # length, points
    ,
    'ytick.major.width':
    1.5  # points
    ,
    'ytick.minor.size':
    6  # length, points
    ,
    "xtick.minor.visible":
    True,
    "ytick.minor.visible":
    True,
    'font.weight':
    'bold',
    'ytick.minor.width':
    1  # points
    ,
    'font.serif': ("Times", "Palatino", "Computer Modern Roman",
                   "New Century Schoolbook", "Bookman"),
    'font.sans-serif':
    ("Helvetica", "Avant Garde", "Computer Modern Sans serif"),
    'font.monospace': ("Courier", "Computer Modern Typewriter"),
    'font.cursive':
    "Zapf Chancery"
})

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import multiprocessing

multiprocessing.cpu_count()

44

In [3]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           377G         13G        358G        970M        5.3G        357G
Swap:          1.0G          0B        1.0G


In [4]:
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import sys
import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp

# ML
import sklearn

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

#Set the display format to be scientific for ease of analysis
# pd.options.display.float_format = '{:,.2g}'.format

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.14.0


In [7]:
# NLP
import gensim 
from gensim import models
from gensim.models import Word2Vec
from gensim import corpora
# import tensorflow 

import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim

In [8]:
try:
    import subprocess
    gitd = os.path.abspath(os.path.join(os.path.dirname(__file__), os.pardir))
    githash = subprocess.check_output('git --git-dir={0:s} --work-tree={1:s} '\
              'rev-parse HEAD'.format(gitd+'/.git',gitd),shell=True).rstrip()
except:
    githash = 'No repo found'

print(githash)

No repo found


In [9]:
# !pip install transformers==2.8.0 --user
# !pip install torch==1.4.0 --user

In [10]:
%load_ext watermark
%watermark -v -h -n -g -m -p jupyerlab,numpy,scipy,sklearn,pandas,matplotlib,nltk,gensim,tensorflow,spacy

Tue Jun 09 2020 

CPython 3.6.2
IPython 6.1.0

jupyerlab not installed
numpy 1.18.1
scipy 1.4.1
sklearn 0.21.3
pandas 0.24.2
matplotlib 2.0.2
nltk 3.5
gensim 3.8.3
tensorflow 1.14.0
spacy 2.2.4

compiler   : GCC 5.4.0
system     : Linux
release    : 3.10.0-1062.9.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 44
interpreter: 64bit
host name  : worker0012
Git hash   : 9f467f89feb15ee1bd2007bc58ac03880df5a02f


In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config



INFO:transformers.file_utils:PyTorch version 1.4.0 available.


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')


In [ ]:
text ="""
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.
"""


preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

In [13]:
from webapp import load_review_helper, NLP_cleaning, NLP_summarization, ldacomplaints

In [14]:
import glob

In [15]:
def load_yelp_files(region):
    region = region.lower()
    if region == 'ny':
        review_files = glob.glob('scraped_data_nodupl_biz/' + '1????*csv')
    elif region == 'sf':
        review_files = glob.glob('scraped_data_nodupl_biz/' + '9????*csv')
    df_new = load_review_helper.read_all_reviews_in_area(review_files)
    df_new.drop_duplicates(inplace=True, keep='first')
    return df_new

df_NY = load_yelp_files('ny')
df_SF = load_yelp_files('sf')
df = pd.concat([df_NY, df_SF])


In [16]:
# del df_new
df_new = pd.read_csv('data_model/cleaned_tokenized_df-2020-06-07.csv')
df_new.head()

,review_name,review_rating,review_date,review_text,biz_url,biz_rating,biz_name,review_text_lem_cleaned,review_text_lem_cleaned_tokenized,review_text_lem_cleaned_tokenized_nostop
0,Lauren D.,5.0,2020-05-12,boyfriend live near walter consistently please...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles,staff month question service store today breat...,"['staff', 'month', 'question', 'service', 'sto...","['staff', 'month', 'question', 'service', 'sto..."
1,Jackie W.,5.0,2019-05-29,dog jojo loves walters labradoodle puppy lots ...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles,lot energy work time place day mean lot exerci...,"['lot', 'energy', 'work', 'time', 'place', 'da...","['lot', 'energy', 'time', 'day', 'lot', 'exerc..."
2,Amelia H.,5.0,2020-01-14,love walter pup sage always crazy excited real...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles,day morning hand staff competent love lunchtim...,"['day', 'morning', 'hand', 'staff', 'competent...","['day', 'morning', 'hand', 'staff', 'competent..."
3,Paulina G.,5.0,2019-04-30,first visit definitely last super professional...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles,visit staff walter style fir aone place trust,"['visit', 'staff', 'walter', 'style', 'fir', '...","['visit', 'staff', 'walter', 'style', 'fir', '..."
4,Kat N.,5.0,2020-03-19,walter amazing beginning corgi quinci loves ev...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles,minute day time pick day staff family,"['minute', 'day', 'time', 'pick', 'day', 'staf...","['minute', 'day', 'time', 'pick', 'day', 'staf..."


In [17]:
df.head()


,review_name,review_rating,review_date,review_text,biz_url,biz_rating,biz_name
0,Lauren D.,5.0,2020-05-12,My boyfriend and I live near Walter's and have...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles
1,Jackie W.,5.0,2019-05-29,Our dog Jojo LOVES walters! We have a labradoo...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles
2,Amelia H.,5.0,2020-01-14,We love Walter's!! My pup Sage is always crazy...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles
3,Paulina G.,5.0,2019-04-30,This is my first visit but it definitely won't...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles
4,Kat N.,5.0,2020-03-19,Walter's has been amazing from the very beginn...,/biz/walters-pet-styles-new-york?osq=pet+sitting,4.5,Walter’s Pet Styles


# bi-directional RNN in our encoding layer, and attention in our decoding layer
similar to Xin Pan’s and Peter Liu’s model from “Sequence-to-Sequence with Attention Model for Text Summarization”

Some cleaning unique to this:
    
- Convert to lowercase.
- Replace contractions with their longer forms.
- Remove any unwanted characters (this step needs to be done after replacing the contractions because apostrophes will be removed. Notice the backward slash before the hyphen. Without this slash, all characters that are ‘between’ the characters before and after the hyphen would be removed. This can create some unwanted effects. To give you an example, typing “a-d” would remove a,b,c,d.).
- Stop words will only be removed from the descriptions. They are not very relevant in training the model, so by removing them we are able to train the model faster because there is less data. They will remain in the summaries because they are rather short and I would prefer for them to sound more like natural phrases.

In [29]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [21]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text,  
                  flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return text

In [ ]:
# Clean the summaries and texts
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

# use pre-trained word vectors/embeddings

ConceptNet Numberbatch (CN). Based on the work of its creators, it seems to outperform GloVe, which makes sense because CN is an ensemble of embeddings that includes GloVe.

limit our vocabulary to words that are either in CN or occur more than 20 times in our dataset. This will allow us to have very good embeddings for every word because the model can better understand how words are related when they see them more times.

In [23]:
!wget https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-en-19.08.txt.gz

--2020-06-09 12:27:17--  https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-en-19.08.txt.gz
Resolving conceptnet.s3.amazonaws.com (conceptnet.s3.amazonaws.com)... 52.216.184.35
Connecting to conceptnet.s3.amazonaws.com (conceptnet.s3.amazonaws.com)|52.216.184.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 325403502 (310M) [application/x-gzip]
Saving to: 'numberbatch-en-19.08.txt.gz'

100%[======================================>] 325,403,502 65.6MB/s   in 4.9s   

2020-06-09 12:27:22 (63.8 MB/s) - 'numberbatch-en-19.08.txt.gz' saved [325403502/325403502]



In [25]:
embeddings_index = {}
with open('CN_embeddings/numberbatch-en-19.08.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

When building your word_embedding_matrix , it is very important that you set its “dtype” of np.zeros to float32. The default value is float64, but this won’t work with TensorFlow because it expects values to be limited to _32.

In [27]:
embedding_dim = 300
nb_words = len(vocab_to_int)
word_embedding_matrix = np.zeros((nb_words, embedding_dim), 
                                 dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

NameError: name 'vocab_to_int' is not defined

To help train the model faster, we are going to sort the reviews by the length of the descriptions from shortest to longest. This will help each batch to have descriptions of similar lengths, which will result in less padding, thus less computing. 